# Modelo de redes com Keras - Simples

Aplicação didática da criação de uma rede com Keras. 

Não fazemos aqui preparação dos dados (normalização p, ex.) e também não separamos dados em treino, validação e teste.

Objetivo do dataset: Prever a direção de PETR4 no dia seguinte baseando-se nos retornos de 5 ativos no dia anterior.

In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import datetime as dt
import numpy as np
from pandas_datareader import data
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [2]:
from numpy import loadtxt
from keras.models import Sequential
from keras.layers import Dense
#import keras

Using TensorFlow backend.


## Criando Dataset

In [3]:
start_date = '2000-01-01'
end_date = '2019-05-01'

In [17]:
def retorno(data):
    return data['Close']/data['Open'] - 1

In [18]:
petr = data.DataReader('PETR4.SA', 'yahoo', start_date, end_date)
painel_ret = pd.DataFrame(index = petr.index)

In [19]:
Tickerlist = ['PETR4.SA', 'VALE3.SA', 'ITUB4.SA', 'CSNA3.SA', 'ABEV3.SA']

In [20]:
N = len(Tickerlist)

In [21]:
for ticker in Tickerlist:
    painel_ret[ticker] = retorno (data.DataReader(ticker, 'yahoo', start_date, end_date))

tickersignal = 'PETR4.SA'+'sign'
painel_ret[tickersignal]  =[1 if painel_ret.shift(-1).loc[i, 'PETR4.SA']>0 else 0 for i in painel_ret.index]
    #signal = [1 if painel_ret.loc[i, ticker]>0 else 0 for i in painel_ret.index]
    #painel_ret[tickersignal] = signal.shift(-1)

In [22]:
painel_ret.tail()

,PETR4.SA,VALE3.SA,ITUB4.SA,CSNA3.SA,ABEV3.SA,PETR4.SAsign
Date,,,,,,
2019-04-25,0.012750,0.001991,0.021199,-0.016271,0.038571,0
2019-04-26,-0.015535,-0.001189,-0.000296,0.024171,0.004945,0
2019-04-29,-0.006534,-0.013545,-0.012371,-0.036667,0.002732,0
2019-04-30,-0.014898,-0.002390,0.004156,0.002759,0.004897,0
2019-05-02,-0.002612,-0.006899,0.010989,-0.021831,-0.016977,0


### Prepare data

In [23]:
painel_ret.isna().sum()

PETR4.SA          0
VALE3.SA          2
ITUB4.SA        261
CSNA3.SA          3
ABEV3.SA          7
PETR4.SAsign      0
dtype: int64

In [24]:
painel_ret = painel_ret.dropna()

In [25]:
painel_ret.head()

,PETR4.SA,VALE3.SA,ITUB4.SA,CSNA3.SA,ABEV3.SA,PETR4.SAsign
Date,,,,,,
2000-12-21,-0.028261,0.000000,0.051515,-0.012756,0.0,1
2000-12-22,0.006711,0.023258,-0.033616,-0.001288,0.0,0
2000-12-26,-0.006667,-0.022221,0.005233,0.028860,0.0,1
2000-12-27,0.007786,0.045453,0.016095,0.013414,0.0,1
2000-12-28,0.015078,0.010870,0.052331,-0.017794,0.0,1


In [26]:
painel_ret.shape

(4482, 6)

In [27]:
# split into input (X) and output (y) variables
X = painel_ret.iloc[:,0:N]
Y = painel_ret.iloc[:,N:]

In [28]:
X.head()

,PETR4.SA,VALE3.SA,ITUB4.SA,CSNA3.SA,ABEV3.SA
Date,,,,,
2000-12-21,-0.028261,0.000000,0.051515,-0.012756,0.0
2000-12-22,0.006711,0.023258,-0.033616,-0.001288,0.0
2000-12-26,-0.006667,-0.022221,0.005233,0.028860,0.0
2000-12-27,0.007786,0.045453,0.016095,0.013414,0.0
2000-12-28,0.015078,0.010870,0.052331,-0.017794,0.0


In [29]:
Y.head()

,PETR4.SAsign
Date,
2000-12-21,1
2000-12-22,0
2000-12-26,1
2000-12-27,1
2000-12-28,1


In [21]:
N

5

## Define Keras Model

### Criando Rede com 4 camadas (uma entrada, dois ocultas e uma de saida)

Utilizamos o método sequential para definir nossa rede de forma sequencia. O comando Dense define rede densa, i.e., cada neuron de uma camada é conectada a todos os neurons
da camada seguinte e da anterior.

- Primeira camada : Input igual ao número de features(no caso N) e com 12 neurons. Função de ativação relu

- Segunda camada:...

- Última camada. Saida com 1 neuron e ativação sigmoid.


In [32]:
# define the keras model
model = Sequential()
model.add(Dense(12, input_dim=N, activation='relu'))
model.add(Dense(8, activation='relu'))
model.add(Dense(5, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

W0930 16:26:38.705261 14368 deprecation_wrapper.py:119] From C:\Users\clayt\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0930 16:26:38.718226 14368 deprecation_wrapper.py:119] From C:\Users\clayt\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0930 16:26:38.722215 14368 deprecation_wrapper.py:119] From C:\Users\clayt\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.



### Compile Keras model

In [ ]:
# compilar o modelo usando a função de erro binary_crossentropy

In [33]:
# compile the keras model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

W0930 16:26:49.462031 14368 deprecation_wrapper.py:119] From C:\Users\clayt\Anaconda3\lib\site-packages\keras\optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

W0930 16:26:49.476018 14368 deprecation_wrapper.py:119] From C:\Users\clayt\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:3376: The name tf.log is deprecated. Please use tf.math.log instead.

W0930 16:26:49.479982 14368 deprecation.py:323] From C:\Users\clayt\Anaconda3\lib\site-packages\tensorflow\python\ops\nn_impl.py:180: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


## Fit
 - epochs = número de vezes que o algoritmo passa sobre o dataset inteiro.
 - batch_size = numero de divisões randomicas do dataset antes que backpropagation ocorra
 - verbose=0 caso não queira que informações apareçam.

In [34]:
model.fit(X, Y, epochs=150, batch_size=10) 

W0930 16:27:01.289830 14368 deprecation_wrapper.py:119] From C:\Users\clayt\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:986: The name tf.assign_add is deprecated. Please use tf.compat.v1.assign_add instead.



Epoch 1/150
4482/4482 [==============================] - 0s 108us/step - loss: 0.6920 - acc: 0.5263
Epoch 2/150
4482/4482 [==============================] - 0s 65us/step - loss: 0.6915 - acc: 0.5308
Epoch 3/150
4482/4482 [==============================] - 0s 71us/step - loss: 0.6914 - acc: 0.5308
Epoch 4/150
4482/4482 [==============================] - 0s 68us/step - loss: 0.6913 - acc: 0.5308
Epoch 5/150
4482/4482 [==============================] - 0s 70us/step - loss: 0.6916 - acc: 0.5308
Epoch 6/150
4482/4482 [==============================] - 0s 69us/step - loss: 0.6915 - acc: 0.5308
Epoch 7/150
4482/4482 [==============================] - 0s 73us/step - loss: 0.6917 - acc: 0.5308
Epoch 8/150
4482/4482 [==============================] - 0s 74us/step - loss: 0.6915 - acc: 0.5308: 0s - loss: 0.6922 - acc: 0.52
Epoch 9/150
4482/4482 [==============================] - 0s 69us/step - loss: 0.6915 - acc: 0.5308
Epoch 10/150
4482/4482 [==============================] - 0s 70us/step - loss

## Perfomance

In [41]:
teste_loss, accuracy = model.evaluate(X, Y)  ## use verbose=0 caso nao queria atualizacoes printadas
print('Accuracy: %.2f' % (accuracy*100))
print('Loss: %.2f' % (teste_loss*100))

4482/4482 [==============================] - 0s 7us/step
Accuracy: 54.57
Loss: 68.52


### Predictions

In [43]:
# make probability predictions with the model
predictions = model.predict(X)
# round predictions 
##Using sigmoid in the output layer, menas that the result will be a probability between zero and 1. We use the following rounding to 
#get values 0 or 1

rounded = [np.around(x, decimals=0) for x in predictions]

In [45]:
X_v = X.values
Y_v = Y.values

In [47]:
# First 10 predicted agains the real
for i in range(10):
	print('%s (expected %s)' % (rounded[i].tolist(), Y_v[i].tolist()))

[1.0] (expected [1])
[0.0] (expected [0])
[1.0] (expected [1])
[0.0] (expected [1])
[0.0] (expected [1])
[0.0] (expected [1])
[0.0] (expected [1])
[0.0] (expected [0])
[0.0] (expected [0])
[0.0] (expected [1])


### Para maiores detalhes
https://www.tensorflow.org/guide/keras/overview